In [2]:
import pandas as pd


class Clue:
    suspects = [
        "Miss Scarlet",
        "Mr. Green",
        "Mrs. White",
        "Mrs. Peacock",
        "Colonel Mustard",
        "Professor Plum",
        # Additional Master Detective Suspects
        "Miss Peach",
        "Sgt. Gray",
        "Monsieur Brunette",
        "Madame Rose",
    ]

    weapons = [
        "Candlestick",
        "Knife",
        "Lead Pipe",
        "Revolver",
        "Rope",
        "Wrench",
        # Additional Master Detective Weapons
        "Horseshoe",
        "Poison",
    ]

    rooms = [
        "Hall",
        "Lounge",
        "Dining Room",
        "Kitchen",
        "Ballroom",
        "Conservatory",
        "Billiard Room",
        "Library",
        "Study",
        # Additional Master Detective Rooms
        "Carriage House",
        "Cloak Room",
        "Trophy Room",
        "Drawing Room",
        "Gazebo",
        "Courtyard",
        "Fountain",
        "Studio",
    ]

    motives = [
        "Revenge",
        "Jealousy",
        "Greed",
        "Blackmail",
        "Power",
        "Cover-up",
        "Betrayal",
        "Obsession",
        "Inheritance",
        "Self-preservation",
    ]

    @staticmethod
    def get_times(start: str, end: str, freq: str) -> list:
        times = (
            (
                pd.date_range(start=start, end="23:59", freq=freq).time.tolist()
                + pd.date_range(start="00:00", end=end, freq=freq).time.tolist()
            )
            if end < start
            else pd.date_range(start=start, end=end, freq=freq).time.tolist()
        )
        return [time.strftime("%I:%M %p") for time in times]

In [129]:
from collections import Counter
import constraint
from itertools import chain, cycle
import random


num_players = 4
suspects = Clue.suspects[:6]
weapons = Clue.weapons[:6]
rooms = Clue.rooms[:6]
motives = []  # Clue.motives[:6]
times = []  # Clue.get_times("21:00", "03:00", "1h")

suspect = random.choice(suspects)
weapon = random.choice(weapons)
room = random.choice(rooms)
motive = random.choice(motives) if motives else None
time = random.choice(times) if times else None

deck = suspects + weapons + rooms + motives + times
unfiltered_deck = deck.copy()
deck.remove(suspect)
deck.remove(weapon)
deck.remove(room)
if motive:
    deck.remove(motive)
if time:
    deck.remove(time)
random.shuffle(deck)
hands = [set(deck[i::num_players]) for i in range(num_players)]
hands.reverse()  # Reverse hands so players with fewer cards go first

for i, hand in enumerate(hands):
    print(f"Player {i + 1}: {hand}")

problem_solution = f"{suspect} with the {weapon} in the {room}"
if time:
    problem_solution += f" at {time}"
if motive:
    problem_solution += f" because of {motive}"
print(f"\nSolution: {problem_solution}")


class NotInHandConstraint(constraint.Constraint):
    def __init__(self, k):
        self.k = k

    def __call__(self, variables, domains, assignments, forwardcheck=False):
        return assignments[next(iter(variables))] != self.k


history = []

for i in cycle(range(num_players)):
    problem = constraint.Problem(constraint.MinConflictsSolver(100_000_000))
    random.shuffle(unfiltered_deck)
    for card in unfiltered_deck:
        problem.addVariable(card, list(reversed((range(-1, num_players)))))
    for card in hands[i]:
        problem.addConstraint(lambda x: x == i, (card,))
    for j, (suggestion, responses) in enumerate(history):
        for k, card in responses.items():
            if card is None:
                pass
                # for suggested_card in suggestion:
                #     problem.addConstraint(constraint.NotInSetConstraint([k]), (suggested_card,))
            elif i == j % num_players:
                problem.addConstraint(lambda x: x == k, (card,))
            else:
                pass
                # def has_one_of(*suggestion_card_owners: int) -> bool:
                #     return any(l == k for l in suggestion_card_owners)

                # problem.addConstraint(has_one_of, suggestion)

    def hand_size(*deck: int) -> bool:
        counter = Counter(deck)
        for key in counter:
            if key == -1 and counter[key] != 3:
                return False
            elif counter[key] != len(hands[key]):
                return False
        return True

    problem.addConstraint(hand_size, unfiltered_deck)

    def has_candidate(*elements: int) -> bool:
        return any(l == -1 for l in elements)

    problem.addConstraint(has_candidate, suspects)
    problem.addConstraint(has_candidate, weapons)
    problem.addConstraint(has_candidate, rooms)
    if motives:
        problem.addConstraint(has_candidate, motives)
    if times:
        problem.addConstraint(has_candidate, times)

    problem_solution = problem.getSolution()
    assert problem_solution is not None, print(problem._constraints)
    print(Counter(problem_solution.values())[-1])
    problem_solution = list(problem_solution.items())
    possible_suspects = [
        key for key, j in problem_solution if j == -1 and key in suspects
    ]
    possible_weapons = [
        key for key, j in problem_solution if j == -1 and key in weapons
    ]
    possible_rooms = [key for key, j in problem_solution if j == -1 and key in rooms]
    possible_motives = [
        key for key, j in problem_solution if j == -1 and key in motives
    ]
    possible_times = [key for key, j in problem_solution if j == -1 and key in times]

    if (
        len(possible_suspects) == len(possible_weapons) == len(possible_rooms) == 1
    ) and (len(possible_motives) <= 1 and len(possible_times) <= 1):
        assert possible_suspects[0] == suspect
        assert possible_weapons[0] == weapon
        assert possible_rooms[0] == room
        if motive:
            assert possible_motives[0] == motive
        if time:
            assert possible_times[0] == time
        print(f"Player {i + 1} wins!")
        break

    suggestion = [
        random.choice(possible_suspects),
        random.choice(possible_weapons),
        random.choice(possible_rooms),
    ]
    if motive:
        suggestion.append(random.choice(possible_motives))
    if time:
        suggestion.append(random.choice(possible_times))

    responses = {}
    for j in chain(range(i + 1, num_players), range(i)):
        responses[j] = None
        for card in suggestion:
            if card in hands[j]:
                responses[j] = card
                break
        if responses[j] is not None:
            break
    history.append((suggestion, responses))

Player 1: {'Candlestick', 'Professor Plum', 'Revolver'}
Player 2: {'Colonel Mustard', 'Mrs. White', 'Hall', 'Mrs. Peacock'}
Player 3: {'Wrench', 'Kitchen', 'Lounge', 'Conservatory'}
Player 4: {'Dining Room', 'Miss Scarlet', 'Rope', 'Knife'}

Solution: Mr. Green with the Lead Pipe in the Ballroom


KeyboardInterrupt: 

In [ ]:
from itertools import chain, cycle
import numpy as np
import pandas as pd
import random


num_players = 6
elements = {
    "suspect": Clue.suspects[:6],
    "weapon": Clue.weapons[:6],
    "room": Clue.rooms[:6],
    # "motive": Clue.motives[:6],
    # "time": Clue.get_times("21:00", "03:00", "1h"),
}
solution = {element: random.choice(values) for element, values in elements.items()}

unfiltered_deck = list(chain(*elements.values()))
deck = unfiltered_deck.copy()
for cards in solution.values():
    deck.remove(cards)
random.shuffle(deck)
hands = [set(deck[i::num_players]) for i in range(num_players)]
hands.reverse()  # Reverse hands so players with fewer cards go first

for i, hand in enumerate(hands):
    print(f"Player {i + 1}: {hand}")

print(f"\nSolution: {solution}")

history = []

index = {card: i for i, card in enumerate(unfiltered_deck)}

# Create a ground truth grid
ground_truth = np.zeros((len(unfiltered_deck), num_players))

# Fill in the ground truth grid based on the hands
for player, hand in enumerate(hands):
    for card in hand:
        ground_truth[index[card], player] = 1


def display_grid(grid: np.ndarray) -> None:
    df = pd.DataFrame(
        grid,
        columns=[f"{i + 1}" for i in range(num_players)],
    ).replace({np.nan: "", 0: "✗", 1: "✓"})
    df.index = pd.MultiIndex.from_tuples(
        [
            (element.capitalize(), card)
            for element in elements
            for card in elements[element]
        ],
        names=["Element", "Card"],
    )
    df.columns.name = "Player"
    print(df)


print("Ground Truth Grid:")
display_grid(ground_truth)

all_but = 0
suggestion_elimination = 0
element_deduction = 0
element_inference = 0
turn = 1

for i in cycle(range(num_players)):
    grid = np.full((len(unfiltered_deck), num_players), np.nan)
    last_grid = grid.copy()

    for card in hands[i]:
        grid[index[card], i] = 1.0
    for j, (suggestion, responses) in enumerate(history):
        j %= num_players
        for k, card in responses.items():
            # if player k did not reveal a card
            if card is None:
                # then all suggested cards must not be in player k's hand
                for suggested_card in suggestion:
                    grid[index[suggested_card], k] = 0.0
            # alternatively if player k revealed a card and we are the player who made the suggestion
            elif i == j:
                # then we know that player k has the revealed card
                grid[index[card], k] = 1.0

    while not np.array_equal(grid, last_grid, equal_nan=True):
        last_grid = grid.copy()
        for j in range(len(unfiltered_deck)):
            # if we have deduced that someone has a card
            if np.nansum(grid[j, :]) == 1:
                # then all other players must not have that card
                grid[j, np.isnan(grid[j, :])] = 0.0
        for j in range(num_players):
            # if we have deduced all the cards in player j's hand
            if np.nansum(grid[:, j]) == len(hands[j]):
                # then all other cards must not be in player j's hand
                grid[np.isnan(grid[:, j]), j] = 0.0
            # alternatively if we have deduced that all but len(hands[j]) cards are not in player j's hand
            elif (grid[:, j] == 0.0).sum() == len(unfiltered_deck) - len(hands[j]):
                # then the remaining cards must be in player j's hand
                grid[np.isnan(grid[:, j]), j] = 1.0
                all_but += 1
        for suggestion, responses in history:
            for k, card in responses.items():
                if (
                    # if player k revealed a card
                    card is not None
                    # and we don't know that any of the suggested cards are in player k's hand
                    and np.nansum(grid[[index[c] for c in suggestion], k]) < 1
                    # but we do know that all but one of the suggested cards are not in player k's hand
                    and (grid[[index[c] for c in suggestion], k] == 0.0).sum()
                    == len(suggestion) - 1
                ):
                    # then the one card that we are unsure of must be in player k's hand
                    grid[
                        index[
                            next(c for c in suggestion if np.isnan(grid[index[c], k]))
                        ],
                        k,
                    ] = 1.0
                    suggestion_elimination += 1
        start = 0
        for cards in elements.values():
            end = start + len(cards)
            element_grid = grid[start:end]
            if (
                # If we are unsure of any cells in the grid for a given game element
                np.isnan(element_grid).sum() > 0
                # but we know where all but one of the cards are
                and np.nansum(element_grid) == len(cards) - 1
            ):
                # then we can fill the remaining unknown cells with 0
                element_grid[np.isnan(element_grid)] = 0
                element_deduction += 1
            if (
                # If we know which card is part of the solution
                np.where(element_grid.sum(axis=0) == 0)[0].size == 1
                # and there are any cards with one unknown cell
                and np.where(np.isnan(element_grid).sum(axis=0) == 1)[0].size > 0
            ):
                # then we can fill the unknown cells with 1
                solvable_rows = element_grid[
                    np.where(np.isnan(element_grid).sum(axis=0) == 1)
                ]
                solvable_rows[np.isnan(solvable_rows)] = 1
                element_inference += 1
            start += len(cards)

    print(f"Player {i + 1} Grid:")
    display_grid(grid)
    np.testing.assert_array_equal(
        grid[~np.isnan(grid)],
        ground_truth[~np.isnan(grid)],
        err_msg="Non-NaN values in grid do not match ground truth",
    )

    accusation = len(np.where(grid.sum(axis=1) == 0)[0]) == len(elements)
    suggestion = []
    start = 0
    for element, cards in elements.items():
        end = start + len(cards)
        if not accusation and random.random() < 0.4 and np.sum(grid[start:end, i]) > 0:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(np.where(grid[start:end, i] == 1)[0] + start)
                ]
            )
        else:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(
                        np.where(
                            (np.sum if accusation else np.nansum)(grid, axis=1)[
                                start:end
                            ]
                            == 0
                        )[0]
                        + start
                    )
                ]
            )
        start += len(cards)

    if accusation:
        print(suggestion)
        print(solution)
        assert all(
            suggestion[i] == solution[element] for i, element in enumerate(elements)
        )
        print(f"Player {i + 1} won on turn {turn}!")
        print(f"# of all but deductions: {all_but}")
        print(f"# of suggestion eliminations: {suggestion_elimination}")
        print(f"# of element deductions: {element_deduction}")
        print(f"# of element inferences: {element_inference}")
        break

    print(f"Player {i + 1} suggests: {suggestion}")

    responses = {}
    for j in chain(range(i + 1, num_players), range(i)):
        responses[j] = None
        suggestion_copy = suggestion.copy()
        random.shuffle(suggestion_copy)
        for card in suggestion_copy:
            if card in hands[j]:
                responses[j] = card
                break
        if responses[j] is not None:
            break
    history.append((suggestion, responses))
    turn += 1

In [4]:
from itertools import chain, cycle
import numpy as np
from ortools.sat.python import cp_model
import pandas as pd
import random


num_players = 4
elements = {
    "suspect": Clue.suspects[:6],
    "weapon": Clue.weapons[:6],
    "room": Clue.rooms[:6],
    # "motive": Clue.motives[:6],
    # "time": Clue.get_times("21:00", "03:00", "1h"),
}
solution = {element: random.choice(values) for element, values in elements.items()}

unfiltered_deck = list(chain(*elements.values()))
deck = unfiltered_deck.copy()
for cards in solution.values():
    deck.remove(cards)
random.shuffle(deck)
hands = [set(deck[i::num_players]) for i in range(num_players)]
hands.reverse()  # Reverse hands so players with fewer cards go first

for i, hand in enumerate(hands):
    print(f"Player {i + 1}: {hand}")

print(f"\nSolution: {solution}")

history = []

index = {card: i for i, card in enumerate(unfiltered_deck)}

# Create a ground truth grid
ground_truth = np.zeros((len(unfiltered_deck), num_players))

# Fill in the ground truth grid based on the hands
for player, hand in enumerate(hands):
    for card in hand:
        ground_truth[index[card], player] = 1


def display_grid(grid: np.ndarray) -> None:
    df = pd.DataFrame(
        grid,
        columns=[f"{i + 1}" for i in range(num_players)],
    ).replace({np.nan: "", 0: "✗", 1: "✓"})
    df.index = pd.MultiIndex.from_tuples(
        [
            (element.capitalize(), card)
            for element in elements
            for card in elements[element]
        ],
        names=["Element", "Card"],
    )
    df.columns.name = "Player"
    print(df)


print("Ground Truth Grid:")
display_grid(ground_truth)

all_but = 0
suggestion_elimination = 0
element_deduction = 0
element_inference = 0
turn = 1


model = cp_model.CpModel()

vars = [
    [model.new_bool_var(f"{j},{k}") for k in range(num_players)]
    for j in range(len(unfiltered_deck))
]

# Enforce that each card i is assigned to at most one player j
for j in range(len(unfiltered_deck)):
    model.add(sum(vars[j]) <= 1)

# Enforce that each player j has exactly len(hands[j]) cards assigned to them
for j, hand in enumerate(hands):
    # Create a list of Boolean indicators where vars[card] == j
    assigned_to_j = [row[j] for row in vars]
    # Sum these indicators using LinearExpr.Sum and set it equal to the number of cards in hand j
    model.add(sum(assigned_to_j) == len(hand))

# Enforce that there are len(elements[element]) - 1 cards assigned to players for each element
start = 0
for cards in elements.values():
    end = start + len(cards)
    # Create a list of Boolean indicators where vars[card] == j
    assigned_to_j = [var for row in vars[start:end] for var in row]
    # Sum these indicators using LinearExpr.Sum and set it equal to the number of cards in hand j
    model.add(sum(assigned_to_j) == len(cards) - 1)
    start = end

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
solver.parameters.max_time_in_seconds = 1


class SolutionCallback(cp_model.CpSolverSolutionCallback):
    def __init__(self) -> None:
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.grid = np.zeros((len(unfiltered_deck), num_players))
        self.num_solutions = 0

    def on_solution_callback(self):
        self.grid += np.array([[self.value(var) for var in row] for row in vars])
        self.num_solutions += 1


for i in cycle(range(num_players)):
    # Add assumptions for the cards in player i's hand
    for card in hands[i]:
        model.add_assumption(vars[index[card]][i])

    # Add assumptions for the cards that were revealed to player i in previous turns
    for j, (_, responses) in enumerate(history):
        j %= num_players
        if i == j:
            for k, card in responses.items():
                if card is not None:
                    model.add_assumption(vars[index[card]][k])

    callback = SolutionCallback()
    status = solver.solve(model, callback)
    assert status == cp_model.OPTIMAL or status == cp_model.FEASIBLE
    model.clear_assumptions()
    grid = callback.grid / callback.num_solutions
    # set every cell that does not equal zero or one to NaN
    grid[(grid != 0) & (grid != 1)] = np.nan
    print(f"Player {i + 1} Grid:")
    display_grid(grid)
    np.testing.assert_array_equal(
        grid[~np.isnan(grid)],
        ground_truth[~np.isnan(grid)],
        err_msg="Non-NaN values in grid do not match ground truth",
    )

    accusation = len(np.where(grid.sum(axis=1) == 0)[0]) == len(elements)
    suggestion = []
    start = 0
    for element, cards in elements.items():
        end = start + len(cards)
        if not accusation and random.random() < 0.4 and np.sum(grid[start:end, i]) > 0:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(np.where(grid[start:end, i] == 1)[0] + start)
                ]
            )
        else:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(
                        np.where(
                            (np.sum if accusation else np.nansum)(grid, axis=1)[
                                start:end
                            ]
                            == 0
                        )[0]
                        + start
                    )
                ]
            )
        start += len(cards)

    if accusation:
        print(suggestion)
        print(solution)
        assert all(
            suggestion[i] == solution[element] for i, element in enumerate(elements)
        )
        print(f"Player {i + 1} won on turn {turn}!")
        break

    print(f"Player {i + 1} suggests: {suggestion}")

    responses = {}
    for j in chain(range(i + 1, num_players), range(i)):
        responses[j] = None
        suggestion_copy = suggestion.copy()
        random.shuffle(suggestion_copy)
        for card in suggestion_copy:
            if card in hands[j]:
                responses[j] = card
                # Everyone knows that player j has at least one of the suggested cards
                model.add_at_least_one([vars[index[card]][j] for card in suggestion])
                break
        if responses[j] is None:
            # Everyone knows that player j does not have any of the suggested cards
            model.add(sum(vars[index[card]][j] for card in suggestion) == 0)
        else:
            break
    history.append((suggestion, responses))
    turn += 1

Player 1: {'Miss Scarlet', 'Mrs. Peacock', 'Rope'}
Player 2: {'Lounge', 'Mrs. White', 'Candlestick', 'Hall'}
Player 3: {'Ballroom', 'Professor Plum', 'Conservatory', 'Revolver'}
Player 4: {'Dining Room', 'Colonel Mustard', 'Wrench', 'Lead Pipe'}

Solution: {'suspect': 'Mr. Green', 'weapon': 'Knife', 'room': 'Kitchen'}
Ground Truth Grid:
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet     ✓  ✗  ✗  ✗
        Mr. Green        ✗  ✗  ✗  ✗
        Mrs. White       ✗  ✓  ✗  ✗
        Mrs. Peacock     ✓  ✗  ✗  ✗
        Colonel Mustard  ✗  ✗  ✗  ✓
        Professor Plum   ✗  ✗  ✓  ✗
Weapon  Candlestick      ✗  ✓  ✗  ✗
        Knife            ✗  ✗  ✗  ✗
        Lead Pipe        ✗  ✗  ✗  ✓
        Revolver         ✗  ✗  ✓  ✗
        Rope             ✓  ✗  ✗  ✗
        Wrench           ✗  ✗  ✗  ✓
Room    Hall             ✗  ✓  ✗  ✗
        Lounge           ✗  ✓  ✗  ✗
        Dining Room      ✗  ✗  ✗  ✓
        Kitchen          ✗  ✗  ✗  ✗
        Ballr